In [86]:
import os
import typing as tp

import cv2
import pandas as pd
import numpy as np

import finetune as FT
import dataset as DS

In [94]:
def cropping_image(img:np.ndarray, bboxes:tp.List[tp.Tuple[np.ndarray, np.ndarray]]):
    result = []
    for bbox in bboxes:
        x0y0, x1y1 = bbox
        x0y0 = np.maximum(x0y0, 0)
        x1y1 = np.maximum(x1y1, 0)
        x0, y0 = x0y0.astype(int)
        x1, y1 = x1y1.astype(int)
        # print(img.shape)
        # print(bbox)
        sub_img = img[y0: y1, x0: x1]
        assert sub_img.size >0, bbox
        result.append(sub_img)
    return result

In [88]:
def plot_bbox(image:np.ndarray, bbox_list:tp.List[tp.Tuple[np.ndarray, np.ndarray]]) -> np.ndarray:
    _image = image.copy()
    for bbox in bbox_list:
        x0y0, x1y1 = bbox
        # print(x0y0, x1y1)
        cv2.rectangle(_image, x0y0.astype(int), x1y1.astype(int), color=(255,0,0), thickness=1)
    return _image

    

In [89]:
model_path = "saved_models/epoch_6701.pth"
model, converter, reader = FT.get_easyocr_recognizer_and_training_converter(['ch_tra'], model_path)
character = ''.join(converter.character[1:])
# print(character)

In [90]:
path_to_raw_data = "./all_data/0_v3_test/"
dataset = DS.OCRDataset(root=path_to_raw_data, character=character, label_max_length=34, rgb=True)
print(len(dataset))
print(dataset.df.index)

153
RangeIndex(start=0, stop=153, step=1)


In [95]:
all_image = []
all_pred_str = []
all_conf = []
all_label = []
all_src = []

image_with_bbox_list = []
image_with_bbox_filename = []

for idx in range(len(dataset)):
    image_pil, label = dataset[idx]
    image = np.array(image_pil)

    ocr_pred = reader.readtext(image)
    if ocr_pred:
        bbox_raw_list = [np.asarray(bbox) for bbox, pred_str, conf in ocr_pred]
        bbox_list = [(np.min(bbox, axis=0), np.max(bbox, axis=0)) for bbox in bbox_raw_list]
        pred_strs = [pred_str for bbox, pred_str, conf in ocr_pred]
        conf_score = [conf for bbox, pred_str, conf in ocr_pred]
        labels = [label]*len(conf_score)
        srcs = [dataset.df.loc[idx, "filename"]]*len(conf_score)
        sub_images = cropping_image(image, bbox_list)
    else:
        sub_images = [image]
        pred_strs = [""]
        conf_score = [None]
        labels = [label]
        srcs = [dataset.df.loc[idx, "filename"]]

    all_image.extend(sub_images)
    all_pred_str.extend(pred_strs)
    all_conf.extend(conf_score)
    all_label.extend(labels)
    all_src.extend(srcs)


    image_with_bbox = plot_bbox(image, bbox_list)
    image_with_bbox_list.append(image_with_bbox)
    image_with_bbox_filename.append(dataset.df.loc[idx, "filename"])
    # plt.imshow(image_with_bbox)

# print(all_pred_str)
# print(all_label)
# print(all_image)

# idx = 0
# for image_pil, label in dataset:
#     print(idx)
#     idx+=1
#     # 

['', '台關', '快', '郵', '台智測', '', '究院', '', '腑新', '部', '猷暨部', '廈', '沙能科技示有', '', '', '', '二', '臺灣智駕測試實驗室', '', '中', '台圍', '新', '快', '仁', '', '高智', '廈', '科場', '機', '淵能科', '', '研究院南院', '', '', '', '交灣高', '內', '', '高鐵台南站', '高', '航化測站市坦', '高發二路', '新道', '', '', '', '', '', '新', '化', '', '速暨中', '', '快速公路', '', '臨停接送區', '', '', '', '化', '測', '', '', '站', '車', '', '陽時接送', '高', '汽機車停車場', '%', '', '阿', '', '新市化', '化', '新', '新', '化', '台南', '', '台南站', '新', '大臺南會展中心', '化', '關智測究', '', '大臺南會展中心', '', '研究院南', '歸', '', '沙綠能科技示範場域', '(', '大', '關', '化', '測', '臺', '', '', '資安暨智慧科技研發大', '關', '', '', '', '臺智駕測試實', '', '高台南', '會', '', '歸仁市區', '內', '口', '歸', '市區', '', '', '大潭', '口', '', '', '', '大潭', '心', '臨停接送區', '', '機踏車停車場', '站心', '汽車停車場', '航大', '新化', '大', '快速公路', '廈', '歸仁市區', '阿', '快速公路', '臺', '歸仁市區', '山', '', '仁', '德', '', '新', '中', '快速公路', '', '沙崙站', '口', '阿', '', '阿蓮', '', '汽車停車場', '航', '快速公路', '', '內', '湖內', '新', '', '快速公路', '大', '', '', '快學南展心', '', '', '資', '', '大臺南會展中心', '化', '', '大', '', '', '台南市區'

In [98]:
output_dir = "./all_data/0_v3_CRAFT_cropped/"
os.makedirs(output_dir, exist_ok=True)

df = pd.DataFrame(columns=["filename", "words", "pred_str", "conf_score", "src_file"])

for idx, (image, label, pred_str, src, conf) in enumerate(zip(all_image, all_label, all_pred_str, all_src, all_conf)):
    filename = f"{idx}.jpg"
    df.loc[idx, "filename"] = filename
    df.loc[idx, "words"] = label
    df.loc[idx, "pred_str"] = pred_str
    # df.loc[idx, "conf_score"] = conf_score
    df.loc[idx, "src_file"] = src

    cv2.imwrite(os.path.join(output_dir, filename), image)

df.to_csv(os.path.join(output_dir, "labels.csv"))

    